<a href="https://colab.research.google.com/github/AlfredIsair/Natural-Language-Processing-Projects/blob/main/Keyword_Extraction/Keyword_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
! pip install -q pyspark==3.4.1 spark-nlp==5.1.2

In [20]:
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types  import StringType, DataType, ArrayType
from pyspark.sql.functions import udf, struct
from pyspark.ml import Pipeline
from IPython.core.display import display, HTML
import re


In [21]:
import sparknlp

from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start()

In [22]:
stopwords = StopWordsCleaner().getStopWords()

In [23]:
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [24]:
document= DocumentAssembler() \
        .setInputCol("text").setOutputCol("document")

sentence= SentenceDetector() \
        .setInputCols("document") \
        .setOutputCol("sentence")

token= Tokenizer()\
        .setInputCols("sentence")\
        .setOutputCol("token") \
        .setContextChars(["(", ")", "?", "!", ".", ","])

keywords= YakeKeywordExtraction() \
        .setInputCols("token")  \
        .setOutputCol("keywords") \
        .setMinNGrams(1) \
        .setMaxNGrams(3) \
        .setNKeywords(20) \
        .setStopWords(stopwords) \
        #.setThreshold() \
        #.setWindowSize()


yake_pipeline = Pipeline(stages=[
    document,
    sentence,
    token,
    keywords
])

In [25]:
empty_df = spark.createDataFrame([['']]).toDF("text")

yake_Model = yake_pipeline.fit(empty_df)

In [31]:
#LightPipeline

light_model = LightPipeline(yake_Model)

text = '''
  google is acquiring data science community kaggle. Sources tell us that google is acquiring kaggle, a platform that hosts data science and machine learning competitions.
  Details about the transaction remain somewhat vague , but given that google is hosting its Cloud Next conference in san francisco this week, the official announcement could come as early as tomorrow.
  Reached by phone, kaggle co-founder ceo anthony goldbloom declined to deny that the acquisition is happening. google itself declined 'to comment on rumors'.
  kaggle, which has about half a million data scientists on its platform, was founded by Goldbloom and Ben Hamner in 2010.
  The service got an early start and even though it has a few competitors like DrivenData, TopCoder and HackerRank, it has managed to stay well ahead of them by focusing on its specific niche.
  The service is basically the de facto home for running data science and machine learning competitions.
  With kaggle, google is buying one of the largest and most active communities for data scientists - and with that, it will get increased mindshare in this community, too (though it already has plenty of that thanks to Tensorflow and other projects).
  kaggle has a bit of a history with google, too, but that's pretty recent. Earlier this month, google and kaggle teamed up to host a $100,000 machine learning competition around classifying YouTube videos.
  That competition had some deep integrations with the google Cloud platform, too. Our understanding is that google will keep the service running - likely under its current name. While the acquisition is probably more about Kaggle's community than technology,
  kaggle did build some interesting tools for hosting its competition and 'kernels', too. On kaggle, kernels are basically the source code for analyzing data sets and developers can share this code on the platform (the company previously called them 'scripts').
  Like similar competition-centric sites, kaggle also runs a job board, too. It's unclear what google will do with that part of the service. According to Crunchbase, kaggle raised $12.5 million (though PitchBook says it's $12.75) since its launch in 2010.
  Investors in kaggle include Index Ventures, SV Angel, Max Levchin, Naval Ravikant, google chief economist Hal Varian, Khosla Ventures and Yuri Milner
'''

light_result = light_model.fullAnnotate(text)[0]

# summary view of the keyword extraction outcome, pairing each recognized keyword(s) with its matching source sentence(s)
[(s.metadata['sentence'], s.result) for s in light_result['sentence']]


[('0', 'google is acquiring data science community kaggle.'),
 ('1',
  'Sources tell us that google is acquiring kaggle, a platform that hosts data science and machine learning competitions.'),
 ('2',
  'Details about the transaction remain somewhat vague , but given that google is hosting its Cloud Next conference in san francisco this week, the official announcement could come as early as tomorrow.'),
 ('3',
  'Reached by phone, kaggle co-founder ceo anthony goldbloom declined to deny that the acquisition is happening.'),
 ('4',
  "google itself declined 'to comment on rumors'. \n  kaggle, which has about half a million data scientists on its platform, was founded by Goldbloom and Ben Hamner in 2010."),
 ('5',
  'The service got an early start and even though it has a few competitors like DrivenData, TopCoder and HackerRank, it has managed to stay well ahead of them by focusing on its specific niche.'),
 ('6',
  'The service is basically the de facto home for running data science and

In [33]:
#lets take the extracted keywords from light_result, organizes them into a Pandas DataFrame, sorts the entries by descending order of relevance, and finally displays the highest ranked 30 results.

import pandas as pd

keys_df = pd.DataFrame([(k.result, k.begin, k.end, k.metadata['score'], k.metadata['sentence'])
                        for k in light_result['keywords']],
                       columns = ['keywords', 'begin', 'end', 'score', 'sentence'])
keys_df['score'] = keys_df['score'].astype(float)

#ordering by relevance
keys_df.sort_values(['sentence', 'score']).head(30)

,keywords,begin,end,score,sentence
5,kaggle,46,51,0.053675,0
0,google,3,8,0.058287,0
2,data,23,26,0.107008,0
3,science,28,34,0.225451,0
71,data science,23,34,0.255856,0
4,community,36,44,0.263910,0
1,acquiring,13,21,0.356494,0
8,kaggle,95,100,0.053675,1
6,google,75,80,0.058287,1
10,data,125,128,0.107008,1
